In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import ast
from nltk.corpus import stopwords
from functools import partial
from gensim.corpora import Dictionary # type: ignore
from gensim.models import LdaModel
from gensim.models import CoherenceModel

In [20]:
df = pd.read_csv(r"D:\tesis_u_sabana\data\db_1\db_1.csv", sep="~", encoding='utf-8')

In [21]:
len(df)

2325242

In [22]:
df.head()

id_contrato                                     nombre_entidad  \
0  co1.pccntr.1358665  sena regional antioquia grupo administrativo c...   
1  co1.pccntr.4243635                              departamento del meta   
2  co1.pccntr.4381704  empresa de servicios publicos de acueducto; al...   
3  co1.pccntr.5334124                                                unp   
4  co1.pccntr.2164831  sena regional norte de santander grupo de apoy...   

   nit_entidad_x                                objeto_del_contrato  \
0      899999034  prestacion de los servicios personales de cara...   
1     8920001488  fortalecer el departamento administrativo de p...   
2      901142488  prestacion de servicios de apoyo a la gestion ...   
3      900475780  prestar los servicios de apoyo a la gestion en...   
4      899999034  54_9119_473 prestar servicios personales de ca...   

                          objeto_del_contrato_limpio  \
0  ['personales', 'caracter', 'temporal', 'interp...   
1  ['fortalecer', 'departamento', 'administrativo...   
2  ['empresa', 'publicos', 'realizacion', 'tratam...   
3  ['misionales', 'desarrolla', 'subdireccion', '...   
4  ['__', 'personales', 'caracter', 'temporal', '...   

                            nombre_del_procedimiento  \
0  apoyo a la gestion como interprete de lengua d...   
1  fortalecer el departamento administrativo de p...   
2  prestacion de servicios de apoyo a la gestion ...   
3                      jaime alberto gomez contreras   
4  instructor area de comercio y ventas - gestion...   

                     nombre_del_procedimiento_limpio estado_contrato  \
0    ['interprete', 'lengua', 'senas', 'colombiana']         cerrado   
1  ['fortalecer', 'departamento', 'administrativo...         cerrado   
2  ['empresa', 'publicos', 'realizacion', 'tratam...      modificado   
3         ['jaime', 'alberto', 'gomez', 'contreras']      modificado   
4  ['instructor', 'area', 'comercio', 'ventas', '...      modificado   

                 departamento         ciudad  ...  \
0                   antioquia         itagui  ...   
1                        meta  villavicencio  ...   
2                   santander         suaita  ...   
3  distrito capital de bogota         bogota  ...   
4          norte de santander         cucuta  ...   

                 nombre_de_la_unidad_de_contratación proveedores_invitados  \
0              contratacion servicios personales ctm                   0.0   
1  departamento administrativo de planeacion depa...                   0.0   
2                                 secretaria general                   0.0   
3                                          contratos                   0.0   
4           centro desarrollo rural y minero- cedrum                   0.0   

  proveedores_con_invitacion_directa visualizaciones_del_procedimiento  \
0                                0.0                               0.0   
1                                0.0                               0.0   
2                                0.0                               0.0   
3                                0.0                               0.0   
4                                0.0                               0.0   

   numero_de_lotes codigo_principal_de_categoria       tipo_de_contrato_y  \
0              0.0                   v1.80111600  prestacion de servicios   
1              0.0                   v1.80111603  prestacion de servicios   
2              0.0                   v1.80111620      decreto 092 de 2017   
3              0.0                   v1.80111701  prestacion de servicios   
4              0.0                   v1.80111600  prestacion de servicios   

  estado_resumen duracion_proceso_dias sin_modificaciones  
0     adjudicado              0.838849               True  
1     adjudicado              0.504738               True  
2     adjudicado              0.806172              False  
3     adjudicado              0.737455              False  
4     adjudicado       

In [23]:
# Convert from strings to lists
df['objeto_del_contrato_limpio'] = df['objeto_del_contrato_limpio'].apply(ast.literal_eval)
df['nombre_del_procedimiento_limpio'] = df['nombre_del_procedimiento_limpio'].apply(ast.literal_eval)

In [24]:
def checkif_stopword(word, stopwords):
    return word not in stopwords

def filter_stopwords(tokens, stopwords):
    return list(filter(partial(checkif_stopword, stopwords=stopwords), tokens))

In [25]:
#añadir mas stopwords
spanish_stopwords = set(stopwords.words('spanish'))
english_stopwords = set(stopwords.words('english'))
other_stopwords = set(["he" , "mas", "si","ser","aunque","hace","tan","solo",
                       "tambien","vez","parece","va","da","hacer","asi","haber",
                       "pues","ver","etc","contrato","servicios","prestacion",
                       "mediante","cuenta","fecha","presente","acuerdo","necesario","contratista",
                        "mil","servicio","requiere","contratos", "supervisor","cumplimiento","modificar",
                        "dar","realizar","objeto","contractual","pesos","modificacion",
                        "conforme","contratacion","gestion","entidad","apoyo","ejecucion",
                        "vigente","normatividad","acta",'integral','area','programas','programa',
                        'proyectos', 'proyecto','direccion','general','nacional','departamento',    
                        'denominado','sistema','oficina','implementacion', 'publica','municipio',
                        'secretaria', 'local','fortalecimiento', 'apoyar','centro',"ps", "pro", "adi", "co", 
                        "corrección", "pccntr", "corregir", "corrige", "actos", "administrativo",
                        "descripcion","contratante","cte","conformidad" ,"realiza" ,"atencion" ,"previo" ,"ajusta",
                        "documentos" ,"modifica" ,"anexos","adjuntos" ,"aporte","actividades" ,"interadministrativo",
                        "atencion","previo","ajusta","administrativa","administrativo","documentos","modifica","obliga",
                        "cd","manera", "adelanta", "grupos", "planes" ,"indole", "mayor",
                        "plan" ,"traves" ,"estrategia" ,"estrategias" , "acciones" ,"accion" ,"promocion",
                        "asistencial" ,"institucional" ,"dentro" ,"procedimientos" ,"diferentes" ,"sur" ,"integrada",
                        "personales" ,"caracter","titulada" ,"complementaria",
                        "san" ,"auxiliar" ,"cundinamarca","antioquia","funcionamiento" ,"publicos" ,"bogota" ,"jose",
                        "contratar" ,"regional","meta" ,"unidades" ,
                        "brindar" ,"asuntos" ,"acompanamiento" ,"relacionados"
                        "administracion" ,"municipal","proceso" ,"territorial" ,"administrativas" ,"subdireccion",
                        "cali" ,"santiago" ,"distrito" ,"ciudadana" ,"bp",
                        "usuarios","organizacion","bienes" ,"interes","presentacion",
                        "subdireccion" ,"relacionadas" ,"formulacion" ,"grupo",
                        "publico" ,"poblacion","esfuerzos" ,"aunar","administrativos" ,"convenio",
                        "ejecutar" ,"acciones" ,"manera","informacion","sistemas" ,"urbana" 
                        "riesgo" ,"instituto" ,"laboral" ,"medellin" ,"actividad" ,"vinculo" ,"laborales",
                        "autonomia","cauca","valle","plena","asistencia" ,"propios","tecnicos" ,"tecnicas",
                        "tecnico","tecnica","tecnologico","tecnologica","tecnologicos","tecnologicas","tecnologias",
                        "santander" ,"realizacion","nivel","norte","sector","bienestar","unidad","operativo" ,"modalidad" ,"distrital" ,"politica" ,"establecidas",
                        "uso","occidente","subsecretaria","alcaldia","fin" ,"lineamientos","funciones","pic" ,"suscrito","requerimiento","cualquier","cualquiera",
                        "rural" ,"zona" ,"espacio" ,"identificacion" ,"sabaneta" ,"urbana",
                        "especializado" ,"dimension" ,"arauca" ,"aseguramiento","competitividad","rurales","sostenibilidad",
                        "soporte" ,"casanare" ,"cargo" ,"covid" ,"yopal",
                        "seguimiento","especializados","revision","actualizacion" ,"inversion",
                        "trabajo" ,"gobierno" ,"boyaca" ,"acceso" ,"tunja","gestor" ,"ibague",
                        "corporacion" ,"autonoma" ,"ciudadano","fomento","estrategico"
                        "participacion","fortalecer" ,"implementar","cucuta" ,"capacidades" ,"desarrollar","alguno" ,"algunos",
                        "demas","competencia","asesora","ciudad","sede" ,"barranquilla","puerto","localidad","santa" ,"directa" ,"c" ,"n",
                        "desarrollar","forma","integrado" ,"barrancabermeja","juan" ,"villavicencio","caldas","pereira","manizales",
                        "maria" ,"andres" ,"cartagena" ,"operativa" ,"integracion" ,"inclusion" ,"ministerio" ,"carlos" ,"rodriguez" ,"lopez"
                        ])
stopword_set = spanish_stopwords.union(english_stopwords,other_stopwords)

df['objeto_del_contrato_limpio'] = df['objeto_del_contrato_limpio'].apply(partial(filter_stopwords, stopwords=stopword_set))
df['nombre_del_procedimiento_limpio'] = df['nombre_del_procedimiento_limpio'].apply(partial(filter_stopwords, stopwords=stopword_set))

In [26]:
df['combined_tokens'] = df.apply(lambda row: row['objeto_del_contrato_limpio'] + row['nombre_del_procedimiento_limpio'], axis=1)

# Prepare documents for LDA
texts = df['combined_tokens'].tolist()
dictionary = Dictionary(texts)

dictionary.filter_extremes(
    no_below=1000,          # palabras que aparecen en al menos 20 documentos
    no_above=0.4,         # no aparecen en más del 50% de los documentos
    keep_n=100000         # mantener hasta las 100,000 palabras más frecuentes
)

corpus = [dictionary.doc2bow(text) for text in texts]

In [27]:
lda_gensim = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=20,          # Entre 20-30 suele ser óptimo para corpus grandes
    passes=5,               # pocas pasadas son suficientes en corpus grandes
    iterations=100,         # suficientes iteraciones para converger
    chunksize=5000,        # buen balance memoria/rendimiento
    alpha=0.05,             # small alpha => each document leans toward fewer topics
    eta='auto',             # optimización automática
    eval_every=None,        # ahorra tiempo evitando evaluaciones intermedias
    random_state=7
)

In [28]:
def get_dominant_topic(lda_model, bow):
    topic_probs = lda_model.get_document_topics(bow)
    return max(topic_probs, key=lambda x: x[1])[0] if topic_probs else None

df['dominant_topic'] = [get_dominant_topic(lda_gensim, doc) for doc in corpus]

In [29]:
coherence_model = CoherenceModel(
    model=lda_gensim,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()
print(f"🧠 Coherence Score (c_v): {coherence_score:.4f}")

🧠 Coherence Score (c_v): 0.4431


In [30]:
# Print all topics with top 10 words each
for idx, topic in lda_gensim.print_topics(num_topics=-1, num_words=5):
    print(f"🔹 Topic #{idx}: {topic}")

🔹 Topic #0: 0.072*"educacion" + 0.030*"humano" + 0.025*"departamental" + 0.025*"talento" + 0.023*"educativa"
🔹 Topic #1: 0.048*"salud" + 0.047*"enfermeria" + 0.044*"hospital" + 0.033*"medico" + 0.031*"policia"
🔹 Topic #2: 0.072*"calidad" + 0.056*"recoleccion" + 0.037*"analisis" + 0.034*"encuesta" + 0.029*"datos"
🔹 Topic #3: 0.079*"control" + 0.060*"ambiental" + 0.060*"riesgo" + 0.055*"vigilancia" + 0.026*"inspeccion"
🔹 Topic #4: 0.098*"mantenimiento" + 0.041*"hacienda" + 0.033*"preventivo" + 0.031*"correctivo" + 0.024*"equipos"
🔹 Topic #5: 0.066*"deporte" + 0.060*"recreacion" + 0.042*"fisica" + 0.025*"deportiva" + 0.023*"logistico"
🔹 Topic #6: 0.038*"arrendamiento" + 0.027*"victimas" + 0.026*"inmueble" + 0.025*"dependencias" + 0.023*"formador"
🔹 Topic #7: 0.097*"seguridad" + 0.049*"administracion" + 0.045*"convivencia" + 0.027*"justicia" + 0.025*"derechos"
🔹 Topic #8: 0.075*"infraestructura" + 0.055*"mejoramiento" + 0.036*"construccion" + 0.029*"mantenimiento" + 0.029*"supervision"
🔹 T

In [31]:
df.head(10)

id_contrato                                     nombre_entidad  \
0  co1.pccntr.1358665  sena regional antioquia grupo administrativo c...   
1  co1.pccntr.4243635                              departamento del meta   
2  co1.pccntr.4381704  empresa de servicios publicos de acueducto; al...   
3  co1.pccntr.5334124                                                unp   
4  co1.pccntr.2164831  sena regional norte de santander grupo de apoy...   
5   co1.pccntr.675738                              dane centro occidente   
6  co1.pccntr.5247834  departamento administrativo nacional de estadi...   
7  co1.pccntr.2624021  agencia para la reincorporacion y la normaliza...   
8  co1.pccntr.4774884                                municipio de guarne   
9  co1.pccntr.3078308                          departamento de santander   

   nit_entidad_x                                objeto_del_contrato  \
0      899999034  prestacion de los servicios personales de cara...   
1     8920001488  fortalecer el departamento administrativo de p...   
2      901142488  prestacion de servicios de apoyo a la gestion ...   
3      900475780  prestar los servicios de apoyo a la gestion en...   
4      899999034  54_9119_473 prestar servicios personales de ca...   
5      899999027  vigencia futura mercado_geih_u_th_tu_ot presta...   
6      899999027  censo_economico_2023_git_ce_bma_th prestacion ...   
7      900477169  prestar; con plena autonomia tecnica y adminis...   
8      890982055  prestacion de servicios de apoyo a la gestion ...   
9      890201235  prestar servicios profesionales como ingeniero...   

                          objeto_del_contrato_limpio  \
0  [temporal, interprete, lengua, senas, colombia...   
1  [planeacion, reporte, fisico, financiero, meta...   
2      [empresa, tratamiento, agua, potable, suaita]   
3  [misionales, desarrolla, evaluacion, riesgo, c...   
4  [__, temporal, planear, orientar, formacion, p...   
5  [vigencia, futura, mercado_geih_u_th_tu_ot, se...   
6  [censo_economico__git_ce_bma_th, requieran, pr...   
7      [operativas, relacionada, beneficios, otorga]   
8                                  [control, guarne]   
9  [ingeniero, civil, infraestructura, asesoria, ...   

                            nombre_del_procedimiento  \
0  apoyo a la gestion como interprete de lengua d...   
1  fortalecer el departamento administrativo de p...   
2  prestacion de servicios de apoyo a la gestion ...   
3                      jaime alberto gomez contreras   
4  instructor area de comercio y ventas - gestion...   
5   dtcoc-cd-d-sp-398-2018 - geih_u - sensibilizador   
6  censo_economico_2023_git_ce_bma_th prestacion ...   
7                prestacion de servicios - tecnico 2   
8      prestacion de servicios de apoyo a la gestion   
9              prestacion de servicios profesionales   

                     nombre_del_procedimiento_limpio estado_contrato  \
0            [interprete, lengua, senas, colombiana]         cerrado   
1  [planeacion, reporte, fisico, financiero, meta...         cerrado   
2      [empresa, tratamiento, agua, potable, suaita]      modificado   
3                 [jaime, alberto, gomez, contreras]      modificado   
4           [instructor, comercio, ventas, mercados]      modificado   
5                [dtcoc, sp, geih_u, sensibilizador]         cerrado   
6  [censo_economico__git_ce_bma_th, requieran, pr...         cerrado   
7                                                 []         cerrado   
8                                                 []       terminado   
9                                                 []      modificado   

                 departamento         ciudad  ...  \
0                   antioquia         itagui  ...   
1                        meta  villavicencio  ...   
2                   santander         suaita  ...   
3  distrito capital de bogota         bogota  ...   
4          norte de santander         cucuta  ...   
5                      caldas      manizales  

In [32]:
# Contar cantidad de filas por cada tema dominante
conteo_temas = df['dominant_topic'].value_counts().sort_index()

# Mostrar el resultado
print("🧾 Cantidad de filas por tema dominante:")
print(conteo_temas)


🧾 Cantidad de filas por tema dominante:
dominant_topic
0     108499
1     143977
2      72922
3     153824
4     112748
5      90454
6      67565
7     107529
8     111026
9     112709
10    164878
11     70950
12     94508
13    104969
14    107109
15    155001
16    103496
17    197950
18     76481
19    168647
Name: count, dtype: int64


In [33]:
df.to_csv(r"D:\tesis_u_sabana\data\db_1\db_1_temas_r3.csv", sep="~", encoding='utf-8', index=False)

In [34]:
df[df["documento_proveedor"] == "900658184"]

id_contrato                                 nombre_entidad  \
704109   co1.pccntr.825729  fondo de desarrollo local de santa fe - fdlsf   
1818968  co1.pccntr.564746          centro nacional de memoria  historica   

         nit_entidad_x                                objeto_del_contrato  \
704109       899999061  interventoria tecnica; administrativa y financ...   
1818968      900492141  fabricar e instalar el museo itinerante de la ...   

                                objeto_del_contrato_limpio  \
704109   [interventoria, financiera, consultoria, compl...   
1818968  [fabricar, instalar, museo, itinerante, memori...   

                                  nombre_del_procedimiento  \
704109   interventoria salon comunal las cruces (presen...   
1818968  licitacion publica mochuelo  (presentacion de ...   

                         nombre_del_procedimiento_limpio estado_contrato  \
704109   [interventoria, salon, comunal, cruces, oferta]       terminado   
1818968                   [licitacion, mochuelo, oferta]       terminado   

                       departamento  ciudad  ...  \
704109   distrito capital de bogota  bogota  ...   
1818968  distrito capital de bogota  bogota  ...   

        proveedores_con_invitacion_directa visualizaciones_del_procedimiento  \
704109                                 0.0                          0.852636   
1818968                                0.0                          0.772036   

        numero_de_lotes codigo_principal_de_categoria  \
704109              0.0                   v1.81101500   
1818968             0.0                   v1.95131600   

              tipo_de_contrato_y estado_resumen duracion_proceso_dias  \
704109             interventoria     adjudicado              0.838849   
1818968  prestacion de servicios     adjudicado              0.704900   

        sin_modificaciones                                    combined_tokens  \
704109               False  [interventoria, financiera, consultoria, compl...   
1818968              False  [fabricar, instalar, museo, itinerante, memori...   

        dominant_topic  
704109               8  
1818968              4  

[2 rows x 71 columns]